In [10]:
# Import statements.
import sagemaker
from sagemaker.tensorflow.estimator import TensorFlow
from dataset import *
from model_3d_bounding_box import *

In [7]:
# Global constants.
BUCKET_NAME = 'sagemaker-4bai-project'
S3_TRAIN_IMAGE_DIR = ('s3://{0}/KITTI/data_object_image_2/training/'
                      'image_2/').format(BUCKET_NAME)
S3_LABEL_DIR = 's3://{0}/KITTI/training/label_2/'.format(BUCKET_NAME)
S3_MODEL_DIR = 's3://{0}/model_3d'.format(BUCKET_NAME)

In [3]:
partition = get_train_only_kitti_partition_s3(S3_TRAIN_IMAGE_DIR)

In [4]:
model = get_model_3d_deepbox()

In [5]:
train_args = DEFAULT_TRAIN_ARGS
train_args['batch_size'] = 1
#train_args['model_checkpoint_filename'] = os.path.join('..', 'model',
#                                                       'model_3d_deepbox.h5')
#history = train_s3(model, partition, S3_TRAIN_IMAGE_DIR, S3_LABEL_DIR)

In [6]:
# From train_s3
train_objs = parse_annotation_s3(partition[TRAIN_KEY], S3_LABEL_DIR)
val_objs = parse_annotation_s3(partition[VAL_KEY], S3_LABEL_DIR)
np.random.shuffle(train_objs)
np.random.shuffle(val_objs)
train_gen = data_gen_s3(S3_TRAIN_IMAGE_DIR, train_objs, train_args['batch_size'])
val_gen = data_gen_s3(S3_TRAIN_IMAGE_DIR, val_objs, train_args['batch_size'])
train_steps = int(np.ceil(len(train_objs) / train_args['batch_size']))
val_steps = int(np.ceil(len(val_objs) / train_args['batch_size']))
callbacks = []
if train_args['use_tensorboard']:
    log_dir = 'logs_{0}'.format(datetime.now())
    tensorboard_callback = TensorBoard(log_dir=log_dir)
    callbacks.append(tensorboard_callback)
if train_args['model_checkpoint_filename']:
    checkpoint_callback = ModelCheckpoint(
        train_args['model_checkpoint_filename'],
        save_best_only=True)
    callbacks.append(checkpoint_callback)

  0%|          | 3/5237 [00:00<03:52, 22.54it/s]

Reading labels from S3.


  0%|          | 3/1496 [00:00<01:08, 21.94it/s]

Reading labels from S3.


100%|██████████| 1496/1496 [01:10<00:00, 21.07it/s]


In [ ]:
history = model.fit(
    x=train_gen,
    epochs=train_args['epochs'],
    callbacks=callbacks,
    validation_data=val_gen,
    steps_per_epoch=train_steps,
    validation_steps=val_steps)

Epoch 1/5
   51/11033 [..............................] - ETA: 3:43:14 - loss: 640.3662 - dense_1_loss: 152.4930 - lambda_loss: 2.8382 - softmax_loss: 7.6889

In [ ]:
# model_3d_train.py
import argparse
from dataset import *
from model_3d_bounding_box import *

BUCKET_NAME = 'sagemaker-4bai-project'
S3_TRAIN_IMAGE_DIR = ('s3://{0}/KITTI/data_object_image_2/training/'
                      'image_2/').format(BUCKET_NAME)
S3_LABEL_DIR = 's3://{0}/KITTI/training/label_2/'.format(BUCKET_NAME)


def parse_args() -> argparse.Namespace:
    """Returns the command line arguments.
    :return: The command line arguments as a Namespace object.
    """
    parser = argparse.ArgumentParser(description='Trains the 3D model.')
    # Hyperparameters.
    parser.add_argument('--epochs', type=int)
    # S3 training configuration parameters.
    parser.add_argument('--model_dir', type=str, help='S3 location where the '
                        'checkpoint data and models can be exported to during '
                        'training.')
    return parser.parse_args()


def train_model(args: argparse.Namespace) -> None:
    """Trains the model using the command line arguments.
    :param args: The command line arguments.
    """
    partition = get_train_only_kitti_partition_s3(S3_TRAIN_IMAGE_DIR)
    model = get_model_3d_deepbox()
    train_args = DEFAULT_TRAIN_ARGS
    if args.epochs:
        train_args['epochs'] = args.epochs
    train_args['batch_size'] = 1
    history = train_s3(model, partition, S3_TRAIN_IMAGE_DIR, S3_LABEL_DIR)


if __name__ == '__main__':
    args = parse_args()
    train_model(args)


In [19]:
estimator = TensorFlow(py_version='py37',
                       framework_version='2.3',
                       model_dir=S3_MODEL_DIR,
                       role=sagemaker.get_execution_role(),
                       instance_count=1,
                       instance_type='ml.t2.medium',
                       entry_point='model_3d_train.py')

In [20]:
estimator.fit()

ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: 1 validation error detected: Value 'ml.t2.medium' at 'resourceConfig.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.p4d.24xlarge, ml.c5n.xlarge, ml.p3.16xlarge, ml.m5.large, ml.p2.16xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.c5.4xlarge, ml.c5n.18xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.c4.8xlarge, ml.g4dn.2xlarge, ml.c5.9xlarge, ml.g4dn.4xlarge, ml.c5.xlarge, ml.g4dn.16xlarge, ml.c4.xlarge, ml.g4dn.8xlarge, ml.c5n.2xlarge, ml.c5n.4xlarge, ml.c5.18xlarge, ml.p3dn.24xlarge, ml.p3.2xlarge, ml.m5.xlarge, ml.m4.10xlarge, ml.c5n.9xlarge, ml.m5.12xlarge, ml.m4.xlarge, ml.m5.24xlarge, ml.m4.2xlarge, ml.p2.8xlarge, ml.m5.2xlarge, ml.p3.8xlarge, ml.m4.4xlarge]